![Image text](картинка.png)

In [9]:
from sympy import randprime, mod_inverse

def generate_rsa_keys(bits=2048):
    # Генерация двух простых чисел p и q (должны быть ~1024 бита каждый)
    p = randprime(2**(bits//2 - 1), 2**(bits//2))
    q = randprime(2**(bits//2 - 1), 2**(bits//2))
    
    # Вычисление модуля n и функции Эйлера
    n = p * q
    phi = (p - 1) * (q - 1)
    
    # Выбор публичной экспоненты и вычисление приватной
    e = 65537
    d = mod_inverse(e, phi)
    
    return (e, n), (d, p, q)

public_key, private_key = generate_rsa_keys()
e, n = public_key
d, p, q = private_key
print(f"Public key: {public_key}")
print(f"Private key: {private_key}")

Public key: (65537, 9888496876385711239364685590879130219362939061590901766271302576999876039756304536147597552941805915283705915559046924014638498612586144478058544650584626608103255353773897733336073914922542854137189095508897163522938333307194596623555262668341940987653712701263967026018249396860214971734883150710114415582236125234668415204159118091976974769936647610823573324958079415911560937145039597222585822612807545767626898674520088794262490131082249502239901626686645942295423559087358570920903796691284360790844719348984348187134424597228068882159408429631505186293841066275159227596401501034949522252439446186494222860253)
Private key: (134271836829510725726088067920767474590095357903317873595142166459743193734518446171136107273187864626256464199703920803219964293686626027603098995751335264332378457044469408012203368736584996046457490823403078960267678148375306644114039828624402906524757601551105547346592613916808906470343050731906111332933374546437625217621506202910047195670776

Этот код гарантирует получение двух 1024-битных простых чисел, чье произведение даст 2048-битный модуль n.

In [10]:
from sympy import randprime, mod_inverse
import timeit

def generate_rsa_keys(bits=2048):
    """Генерация RSA ключей для демонстрации"""
    p = randprime(2**(bits//2 - 1), 2**(bits//2))
    q = randprime(2**(bits//2 - 1), 2**(bits//2))
    n = p * q
    phi = (p - 1) * (q - 1)
    e = 65537
    d = mod_inverse(e, phi)
    return (e, n), (d, p, q)

def classic_decrypt(c, d, n):
    """
    Классическая расшифровка RSA: m = c^d mod n
    
    Args:
        c: шифротекст (ciphertext)
        d: приватная экспонента (private exponent) 
        n: модуль RSA (modulus)
    
    Returns:
        m: расшифрованное сообщение (plaintext)
    """
    return pow(c, d, n)

# Демонстрация работы
if __name__ == "__main__":
    # Генерируем ключи
    public_key, private_key = generate_rsa_keys()
    e, n = public_key
    d, p, q = private_key
    
    # Тестовое сообщение
    original_message = 42
    print(f"Исходное сообщение: {original_message}")
    
    # Шифрование: c = m^e mod n
    ciphertext = pow(original_message, e, n)
    print(f"Шифротекст: {ciphertext}")
    
    # Расшифровка классическим методом: m = c^d mod n
    decrypted_message = classic_decrypt(ciphertext, d, n)
    print(f"Расшифрованное сообщение: {decrypted_message}")
    
    # Проверка
    print(f"Успех: {original_message == decrypted_message}")

    # Замеряем время выполнения 100 раз
    time_taken = timeit.timeit(
        lambda: classic_decrypt(ciphertext, d, n),
        number=100
    )
    
    print(f"Время для 100 выполнений: {time_taken:.4f} секунд")
    print(f"Среднее время на операцию: {time_taken/100:.6f} секунд")


Исходное сообщение: 42
Шифротекст: 7938166866372272888671904493896083462761297027665219308123965770704189557182173169960013899180732831057157554080585509526021091984947733102079380332533266107246416713875956213333674878979876150153430613328911331312796531281454023406022284517901462805541497274448362119642945405596786961910426538898350187260307601496301097373626678274280248608751748436684698367001700047407102202288748923480158951685038670662249617798858176676202633407361613223309571412195804505313054279525590251865320900484994358903949950924578921775177943390799867397334362167995736441677556992059623862829665094434181667056984159480488627265245
Расшифрованное сообщение: 42
Успех: True
Время для 100 выполнений: 1.6295 секунд
Среднее время на операцию: 0.016295 секунд


In [11]:
def optimized_decrypt(c, d, p, q):
    """
    Оптимизированная расшифровка через КТО
    """
    # mp = c^(d mod (p-1)) mod p
    mp = pow(c, d % (p-1), p)
    
    # mq = c^(d mod (q-1)) mod q
    mq = pow(c, d % (q-1), q)
    
    # m = КТО(mp, mq, p, q)
    n = p * q
    q_inv = pow(q, -1, p)
    m = (mp * q * q_inv + mq * p * pow(p, -1, q)) % n
    
    return m

Формула Китайской теоремы об остатках:

m = (mp × q × (q⁻¹ mod p) + mq × p × (p⁻¹ mod q)) mod n

In [12]:
from sympy import randprime, mod_inverse
import timeit

def generate_rsa_keys(bits=2048):
    # Генерация двух простых чисел p и q
    p = randprime(2**(bits//2 - 1), 2**(bits//2))
    q = randprime(2**(bits//2 - 1), 2**(bits//2))
    
    # Вычисление модуля n и функции Эйлера
    n = p * q
    phi = (p - 1) * (q - 1)
    
    # Выбор публичной экспоненты и вычисление приватной
    e = 65537
    d = mod_inverse(e, phi)
    
    return (e, n), (d, p, q)

def optimized_decrypt(c, d, p, q):
    """
    Оптимизированная расшифровка через КТО
    """
    # mp = c^(d mod (p-1)) mod p
    mp = pow(c, d % (p-1), p)
    
    # mq = c^(d mod (q-1)) mod q
    mq = pow(c, d % (q-1), q)
    
    # m = КТО(mp, mq, p, q)
    n = p * q
    q_inv = pow(q, -1, p)
    m = (mp * q * q_inv + mq * p * pow(p, -1, q)) % n
    
    return m


# Демонстрация работы
if __name__ == "__main__":
    # Генерируем ключи
    public_key, private_key = generate_rsa_keys()
    e, n = public_key
    d, p, q = private_key
    
    # Тестовое сообщение
    original_message = 42
    print(f"Исходное сообщение: {original_message}")
    
    # Шифрование: c = m^e mod n
    ciphertext = pow(original_message, e, n)
    print(f"Шифротекст: {ciphertext}")
    
    # Расшифровка оптимизированным методом
    decrypted_message = optimized_decrypt(ciphertext, d, p, q)
    print(f"Расшифрованное сообщение: {decrypted_message}")
    
    # Проверка
    print(f"Успех: {original_message == decrypted_message}")

    # Замеряем время выполнения 100 раз
    time_taken = timeit.timeit(
        lambda: classic_decrypt(ciphertext, d, n),
        number=100
    )
    
    print(f"Время для 100 выполнений: {time_taken:.4f} секунд")
    print(f"Среднее время на операцию: {time_taken/100:.6f} секунд")

Исходное сообщение: 42
Шифротекст: 2519807455025097055571525621229213991767027742184314190862913836249669054376559935054576066179308636171943965154263420968675164316650526830138420247034041262032457002231031158176735764514811679938690970247773554473328179995113990687151321871289447162562011290616155765146590242103400143942236917410349325420854618706870365819175966725776712093045300114895115686124228228036758957341008565376204761358820893828129641031634237387386943366151633715627430996295312360281691785149106155229203118907245415773579003747132412752315725195857223240770155289246375672999012769248423420608450859941902140489785238963924358353411
Расшифрованное сообщение: 42
Успех: True
Время для 100 выполнений: 1.6284 секунд
Среднее время на операцию: 0.016284 секунд


# Ответ Google AI:

# Использование КТО в OpenSSL для оптимизации RSA

КТО (Китайская теорема об остатках) применяется в OpenSSL для значительного ускорения операций RSA-алгоритма, включая расшифрование и генерацию цифровых подписей, путем разбиения сложных вычислений на более простые компоненты.

## Принцип работы и преимущества

### Большой модуль
- В RSA используется открытый модуль **n**, представляющий собой произведение двух больших секретных простых чисел:  
  `n = p × q`

### Сложность вычислений
- Операции шифрования и расшифрования требуют возведения в степень по модулю **n**
- Работа с огромными числами **n** (тысячи бит) требует значительных вычислительных ресурсов

### Разделение задачи через КТО
КТО позволяет заменить одну сложную операцию по большому модулю **n** на две независимые, параллельно выполняемые операции по меньшим модулям **p** и **q**

### Ускорение вычислений
- Вычисления по модулям **p** и **q** (примерно вдвое меньшим по размеру, чем **n**) выполняются значительно быстрее
- Результаты двух меньших вычислений эффективно объединяются с помощью КТО для получения итогового результата

## Практический результат

Использование КТО в OpenSSL приводит к:
- **Увеличению производительности операций RSA в 3-4 раза**
- Критическому улучшению производительности для высоконагруженных серверов (веб-серверы с HTTPS)
- Обработке миллионов защищенных соединений с оптимальной скоростью
- Прямому влиянию на общую производительность криптографической системы

In [19]:
from sympy import randprime, mod_inverse
import timeit
import random

def generate_rsa_keys(bits=2048):
    """Генерация RSA ключей"""
    p = randprime(2**(bits//2 - 1), 2**(bits//2))
    q = randprime(2**(bits//2 - 1), 2**(bits//2))
    n = p * q
    phi = (p - 1) * (q - 1)
    e = 65537
    d = mod_inverse(e, phi)
    return (e, n), (d, p, q)

def classic_decrypt(c, d, n):
    """Классическая расшифровка RSA: m = c^d mod n"""
    return pow(c, d, n)

def optimized_decrypt(c, d, p, q):
    """
    Оптимизированная расшифровка через КТО
    """
    # mp = c^(d mod (p-1)) mod p
    mp = pow(c, d % (p-1), p)
    
    # mq = c^(d mod (q-1)) mod q
    mq = pow(c, d % (q-1), q)
    
    # m = КТО(mp, mq, p, q)
    n = p * q
    q_inv = pow(q, -1, p)
    m = (mp * q * q_inv + mq * p * pow(p, -1, q)) % n
    
    return m

def compare_methods(tests=10, messages=100):
    results = []
    
    for test in range(tests):
        public, private = generate_rsa_keys()
        e, n = public
        d, p, q = private
        
        # Подготовка данных
        texts = [random.randint(2, n-1) for _ in range(messages)]
        encrypted = [pow(msg, e, n) for msg in texts]
        
        # Замер времени
        time_classic = timeit.timeit(
            lambda: [classic_decrypt(ct, d, n) for ct in encrypted], number=1)
        
        time_optimized = timeit.timeit(
            lambda: [optimized_decrypt(ct, d, p, q) for ct in encrypted], number=1)
        
        results.append((time_classic, time_optimized))
        print(f"Тест {test+1}: классика {time_classic:.3f}с, КТО {time_optimized:.3f}с")
    
    # Итоги
    avg_classic = sum(t[0] for t in results) / tests
    avg_optimized = sum(t[1] for t in results) / tests
    wins = sum(1 for t in results if t[1] < t[0])
    
    print(f"\nИтоги: КТО быстрее в {wins}/{tests} случаях")
    print(f"Ускорение: в {avg_classic/avg_optimized:.1f} раза")

if __name__ == "__main__":
    compare_methods()


Тест 1: классика 1.615с, КТО 0.479с
Тест 2: классика 1.601с, КТО 0.469с
Тест 3: классика 1.639с, КТО 0.485с
Тест 4: классика 1.666с, КТО 0.488с
Тест 5: классика 1.642с, КТО 0.497с
Тест 6: классика 1.678с, КТО 0.491с
Тест 7: классика 1.730с, КТО 0.494с
Тест 8: классика 1.698с, КТО 0.504с
Тест 9: классика 1.713с, КТО 0.486с
Тест 10: классика 1.664с, КТО 0.514с

Итоги: КТО быстрее в 10/10 случаях
Ускорение: в 3.4 раза
